In [ ]:
# Mount Google Drive
from google.colab import drive
import os

# Disable Weights & Biases tracking
os.environ["WANDB_DISABLED"] = "true"

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define directories and file paths
main_dir = "/content/drive/MyDrive/ITCS6156_BERT"
result_dir = os.path.join(main_dir, "Results")
data_dir = os.path.join(main_dir, "Data")
train_path = os.path.join(data_dir, "train_data.txt")
test_path = os.path.join(data_dir, "test_data.txt")
test_solution_path = os.path.join(data_dir, "test_data_solution.txt")


Mounted at /content/drive


In [ ]:
import pandas as pd

# Load training and test data
train_data = pd.read_csv(train_path, sep=":::", engine="python", header=None, names=["ID", "TITLE", "GENRE", "DESCRIPTION"])
test_data = pd.read_csv(test_path, sep=":::", engine="python", header=None, names=["ID", "TITLE", "DESCRIPTION"])
test_solution = pd.read_csv(test_solution_path, sep=":::", engine="python", header=None, names=["ID", "TITLE", "GENRE", "DESCRIPTION"])

# Combine title and description for text input
train_data["TEXT"] = train_data["TITLE"] + " " + train_data["DESCRIPTION"]
test_data["TEXT"] = test_data["TITLE"] + " " + test_data["DESCRIPTION"]

# Encode genres to numeric labels
train_data["GENRE_LABEL"] = train_data["GENRE"].factorize()[0]
genre_mapping = dict(enumerate(train_data["GENRE"].factorize()[1]))

print("Genre Mapping:", genre_mapping)


Genre Mapping: {0: ' drama ', 1: ' thriller ', 2: ' adult ', 3: ' documentary ', 4: ' comedy ', 5: ' crime ', 6: ' reality-tv ', 7: ' horror ', 8: ' sport ', 9: ' animation ', 10: ' action ', 11: ' fantasy ', 12: ' short ', 13: ' sci-fi ', 14: ' music ', 15: ' adventure ', 16: ' talk-show ', 17: ' western ', 18: ' family ', 19: ' mystery ', 20: ' history ', 21: ' news ', 22: ' biography ', 23: ' romance ', 24: ' game-show ', 25: ' musical ', 26: ' war '}


In [ ]:
from sklearn.model_selection import train_test_split

# Split training data into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data["TEXT"], train_data["GENRE_LABEL"], test_size=0.2, random_state=42
)


In [ ]:
from transformers import DistilBertTokenizer

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize text data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(test_data["TEXT"]), truncation=True, padding=True, max_length=512)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
import tensorflow as tf

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(16)


In [ ]:
from transformers import TFDistilBertForSequenceClassification

# Load pre-trained DistilBERT model for sequence classification
num_labels = len(genre_mapping)
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=7e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3  # Adjust based on dataset size
)


Epoch 1/3
2711/2711 [==============================] - 2580s 943ms/step - loss: 1.3281 - sparse_categorical_accuracy: 0.6037 - val_loss: 1.1617 - val_sparse_categorical_accuracy: 0.6383
Epoch 2/3
2711/2711 [==============================] - 2557s 943ms/step - loss: 0.8537 - sparse_categorical_accuracy: 0.7424 - val_loss: 1.2575 - val_sparse_categorical_accuracy: 0.6333
Epoch 3/3
2711/2711 [==============================] - 2555s 943ms/step - loss: 0.5416 - sparse_categorical_accuracy: 0.8387 - val_loss: 1.4427 - val_sparse_categorical_accuracy: 0.6332


In [ ]:
# Evaluate the model
results = model.evaluate(val_dataset)
print("Validation Loss and Accuracy:", results)


678/678 [==============================] - 207s 305ms/step - loss: 1.4427 - sparse_categorical_accuracy: 0.6332
Validation Loss and Accuracy: [1.4427106380462646, 0.6332195997238159]


In [ ]:
# Convert test_encodings to TensorFlow-compatible format
test_dataset = tf.data.Dataset.from_tensor_slices(dict(test_encodings)).batch(16)

# Predict on test data
test_predictions = model.predict(test_dataset)
test_predicted_labels = tf.argmax(test_predictions.logits, axis=1).numpy()

# Clean up genre mapping
clean_genre_mapping = {k: v.strip() for k, v in genre_mapping.items()}

# Map numeric predictions to genre names
test_data["PREDICTED_GENRE_LABEL"] = test_predicted_labels
test_data["PREDICTED_GENRE"] = test_data["PREDICTED_GENRE_LABEL"].map(clean_genre_mapping)

# Debugging output
print("Predicted Labels (first 20):", test_predicted_labels[:20])
print("Mapped Genres (first 20):", test_data["PREDICTED_GENRE"].head(20))

# Save predictions to CSV
predictions_file = os.path.join(result_dir, "predictions.csv")
test_data[["ID", "TITLE", "PREDICTED_GENRE"]].to_csv(predictions_file, index=False)
print(f"Predictions saved to {predictions_file}")


3388/3388 [==============================] - 1030s 304ms/step
Predicted Labels (first 20): [ 7  0  3  0  0  1  4  4  3  0  1  0  0  0 12  3  4 17  3  0]
Mapped Genres (first 20): 0          horror
1           drama
2     documentary
3           drama
4           drama
5        thriller
6          comedy
7          comedy
8     documentary
9           drama
10       thriller
11          drama
12          drama
13          drama
14          short
15    documentary
16         comedy
17        western
18    documentary
19          drama
Name: PREDICTED_GENRE, dtype: object
Predictions saved to /content/drive/MyDrive/ITCS6156_BERT/Results/predictions.csv


In [ ]:
# Inspect predicted labels
print("Predicted labels (first 20):", test_predicted_labels[:20])

# Print genre mapping
print("Genre Mapping:", genre_mapping)


Predicted labels (first 20): [ 7  0  3  0  0  1  4  4  3  0  1  0  0  0 12  3  4 17  3  0]
Genre Mapping: {0: ' drama ', 1: ' thriller ', 2: ' adult ', 3: ' documentary ', 4: ' comedy ', 5: ' crime ', 6: ' reality-tv ', 7: ' horror ', 8: ' sport ', 9: ' animation ', 10: ' action ', 11: ' fantasy ', 12: ' short ', 13: ' sci-fi ', 14: ' music ', 15: ' adventure ', 16: ' talk-show ', 17: ' western ', 18: ' family ', 19: ' mystery ', 20: ' history ', 21: ' news ', 22: ' biography ', 23: ' romance ', 24: ' game-show ', 25: ' musical ', 26: ' war '}


In [ ]:
# Save the trained model and tokenizer
model.save_pretrained(os.path.join(result_dir, "distilbert_genre_model"))
tokenizer.save_pretrained(os.path.join(result_dir, "distilbert_genre_model"))


('/content/drive/MyDrive/ITCS6156_BERT/Results/distilbert_genre_model/tokenizer_config.json',
 '/content/drive/MyDrive/ITCS6156_BERT/Results/distilbert_genre_model/special_tokens_map.json',
 '/content/drive/MyDrive/ITCS6156_BERT/Results/distilbert_genre_model/vocab.txt',
 '/content/drive/MyDrive/ITCS6156_BERT/Results/distilbert_genre_model/added_tokens.json')